In [1]:
!pip install langchain langchain_community  pinecone langchain_pinecone openai langchain-openai langchain-classic

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-classic to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is still looking at multiple versions of langchain-classic to determine which version is compatible with other requirements. T

In [5]:
from google.colab import userdata

PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
DEPLOYMENT_NAME = userdata.get('DEPLOYMENT_NAME')

In [2]:
from google.colab import files

uploaded = files.upload()

filename = list(uploaded.keys())[0]

with open(filename, 'r') as f:
    text = f.read()

from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_text(text)

print(len(texts))


Saving churchill_speech.txt to churchill_speech (1).txt
23


In [3]:
!pip install langchain_openai

In [6]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    deployment="text-embedding-3-small",
    model="text-embedding-3-small",
    openai_api_type="azure",
    openai_api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_version="2023-05-15",
    chunk_size=2048
)


In [7]:
vector = embeddings.embed_query(texts[0])
print(len(vector))
print(vector)

1536
[0.0414695143699646, 0.04774028807878494, 0.0648132786154747, -0.006053358782082796, -0.02186761610209942, -0.014326663687825203, 0.004025083035230637, 0.003415742190554738, -0.01968199759721756, 0.024282094091176987, 0.03879184648394585, -0.026868216693401337, -0.004439892712980509, -0.025449281558394432, 0.0027763633988797665, 0.036938074976205826, 0.017462052404880524, 0.008485001511871815, 0.01965911313891411, 0.033848460763692856, -0.036640558391809464, 0.015448080375790596, -0.050395071506500244, -0.011202719062566757, -0.03551914170384407, -0.0017093014903366566, -0.004685917869210243, -0.0649048238992691, 0.00784419197589159, 0.00062900863122195, 0.04320885241031647, -0.013067931868135929, -0.02050589583814144, 0.0297976303845644, 0.003902070689946413, -0.05941217020153999, 0.010910921730101109, -0.014589853584766388, -0.03130811080336571, 0.027692114934325218, -0.017462052404880524, -0.03675498813390732, 0.011454465799033642, 0.033322080969810486, 0.029889173805713654, -0

In [8]:
import pinecone

from langchain_pinecone import Pinecone
from langchain_classic.vectorstores import Pinecone


pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)

In [9]:
from  pinecone import ServerlessSpec

index_name = "churchill-speech"

if index_name not in pc.list_indexes().names():
  pc.create_index(
      name=index_name,
      dimension=1536,
      metric="cosine",
      spec = ServerlessSpec(
          cloud = "aws",
          region = "us-east-1"
      )
  )

In [13]:
import os
#from pinecone import Pinecone as PineconeClient
from langchain_pinecone import Pinecone
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY


chunk =  text_splitter.create_documents([text])


vectorstore = Pinecone.from_documents(
    documents=chunk,
    embedding=embeddings,
    index_name=index_name
)

# ✅ Describe index stats
index = pc.Index(index_name)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 115}},
 'total_vector_count': 115,
 'vector_type': 'dense'}


In [14]:
query = 'where should we fight ?'
result = vectorstore.similarity_search(query)
print(result)

[Document(metadata={}, page_content="streets, we shall fight in the hills; we shall never surrender, and even if, which I do not for a moment\nbelieve, this Island or a large part of it were subjugated and starving, then our Empire beyond the\nseas, armed and guarded by the British Fleet, would carry on the struggle, until, in God's good time,\nthe New World, with all its power and might, steps forth to the rescue and the liberation of the old."), Document(metadata={}, page_content="streets, we shall fight in the hills; we shall never surrender, and even if, which I do not for a moment\nbelieve, this Island or a large part of it were subjugated and starving, then our Empire beyond the\nseas, armed and guarded by the British Fleet, would carry on the struggle, until, in God's good time,\nthe New World, with all its power and might, steps forth to the rescue and the liberation of the old."), Document(metadata={}, page_content="streets, we shall fight in the hills; we shall never surrende

In [15]:
from langchain_openai import AzureChatOpenAI
from google.colab import userdata
AZURE_OPENAI_API_KEY= userdata.get('AZURE_OPENAI_KEY')
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment=DEPLOYMENT_NAME,
    api_version="2024-05-01-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # organization="...",
    # model="gpt-35-turbo",
    # model_version="0125",
    # other params...
)

In [16]:
from langchain_classic.chains import RetrievalQA

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":5})

chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [17]:
query = 'Answer only from the provided input. where shoul we fight ?'
answer = chain.invoke(query)
print(answer)

ValueError: Azure has not provided the response due to a content filter being triggered

In [18]:
query = 'who is the king of belgium at that time?'
answer = chain.invoke(query)
print(answer)

{'query': 'who is the king of belgium at that time?', 'result': 'The King of Belgium at that time was **King Leopold III**.'}


In [19]:
query = 'what about the french armies?'
answer = chain.run(query)
print(answer)

/tmp/ipython-input-3011605181.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  answer = chain.run(query)


The French armies were intended to advance across the Somme in great strength to support Belgium and counter the German offensive. However, the German forces executed a rapid and overwhelming mechanized attack, cutting off communications between the French armies and the northern forces. This German advance severed supply lines for food and ammunition and isolated the French armies from their allies. The French armies were unable to effectively respond to the swift and coordinated German assault, which swept through key locations like Amiens, Abbeville, Boulogne, Calais, and almost reached Dunkirk.


In [20]:
query = 'who is the prime minister of india at that time?'
answer = chain.invoke(query)
print(answer)

{'query': 'who is the prime minister of india at that time?', 'result': "I don't know."}


In [21]:
query = 'hello'
answer = chain.invoke(query)
print(answer)

{'query': 'hello', 'result': 'Hello! How can I assist you today?'}


In [22]:
query = 'who is the prime minister of india at that time?'

result = vectorstore.similarity_search_with_score(query , k=10)

filter = [ doc for doc , score in result if score > 0.2]

print(len(filter))

k = len(filter)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":max(k , 3)})

chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

answer = chain.invoke(query)
print(answer)

10
{'query': 'who is the prime minister of india at that time?', 'result': "I don't know. The context provided does not mention the time period or the Prime Minister of India."}


In [23]:
query = 'what about the french armies?'
answer = chain.run(query)
print(answer)

The context mentions the French armies in relation to the German offensive during World War II. It states that the French had planned to advance across the Somme in great strength to support Belgium, but the German forces launched a rapid and overwhelming attack. This German offensive, involving eight or nine armored divisions, cut off communications between the Allied forces, including the French armies, and disrupted their supply lines for food and ammunition. The German forces advanced through key locations like Amiens, Abbeville, Boulogne, Calais, and almost reached Dunkirk, effectively isolating the French armies from the British forces.

Additionally, the context briefly mentions the "great French Army" in the context of air defense, but does not provide further details about their specific actions or outcomes during this period.
